In [1]:
from src.structure.quantizer import PdbQuantizer
processor = PdbQuantizer(structure_vocab_size=2048) # can be 20, 128, 512, 1024, 2048, 4096
result = processor("data/example_pdb/p1.pdb", return_residue_seq=False)

/home/tanyang/miniconda3/envs/protssn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/tanyang/workspace/ProSST-R/src/structure/quantizer.py:515: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`

In [2]:
result

[407,
 998,
 1841,
 1421,
 653,
 450,
 117,
 822,
 1082,
 70,
 1924,
 1559,
 1831,
 370,
 968,
 1508,
 524,
 1766,
 1490,
 793,
 1079,
 1079,
 531,
 257,
 225,
 300,
 15,
 13,
 677,
 381,
 417,
 302,
 1999,
 139,
 186,
 598,
 598,
 186,
 1242,
 186,
 826,
 68,
 508,
 1788,
 1421,
 676,
 1696,
 944,
 407,
 709,
 736,
 511,
 689,
 1843,
 743,
 43,
 643,
 1440,
 12,
 2020,
 1976,
 1214,
 1973,
 1509,
 1174,
 709,
 1129,
 923,
 1082,
 1484,
 1182,
 844,
 521,
 521,
 1841]

In [38]:
def read_multi_fasta(file_path):
    """
    params:
        file_path: path to a fasta file
    return:
        a dictionary of sequences
    """
    sequences = {}
    current_sequence = ''
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if line.startswith('>'):
                if current_sequence:
                    sequences[header] = current_sequence.upper()
                    current_sequence = ''
                header = line
            else:
                current_sequence += line
        if current_sequence:
            sequences[header] = current_sequence
    return sequences

In [53]:
seqs = list(read_multi_fasta("data/proteingym_v1/residue_alignment/A0A1I9GEU1_NEIME_Kennouche_2019.a2m").keys())
seqs

['>A0A1I9GEU1_NEIME/1-161',
 '>UniRef100_UPI0018A25760/3-135',
 '>UniRef100_UPI00145ECBA7/3-130',
 '>UniRef100_Q00045/8-162',
 '>UniRef100_A0A7U1LX12/3-136',
 '>UniRef100_UPI000BA48856/3-130',
 '>UniRef100_UPI0005E9AABB/13-136',
 '>UniRef100_UPI0013E0C905/9-133',
 '>UniRef100_UPI0018C26155/1-133',
 '>UniRef100_D6HB30/9-137',
 '>UniRef100_B0FXJ6/2-160',
 '>UniRef100_UPI0005E0CA5C/8-166',
 '>UniRef100_UPI000C3233B7/8-166',
 '>UniRef100_UPI000C332612/8-166',
 '>UniRef100_UPI000766DABE/8-165',
 '>UniRef100_UPI000C323150/8-166',
 '>UniRef100_UPI000C34031D/8-165',
 '>UniRef100_UPI000C33F616/8-166',
 '>UniRef100_P57039/8-168',
 '>UniRef100_UPI000C326753/8-166',
 '>UniRef100_UPI000C33E39E/8-166',
 '>UniRef100_UPI000C330815/8-165',
 '>UniRef100_UPI000C321D0A/8-166',
 '>UniRef100_A0A0G4BXR5/8-166',
 '>UniRef100_UPI000766626E/8-166',
 '>UniRef100_UPI0018A26FAC/12-140',
 '>UniRef100_UPI000E57AB74/8-165',
 '>UniRef100_UPI000FC9B0E8/8-165',
 '>UniRef100_A0A807D5K8/8-165',
 '>UniRef100_UPI000E58367C/

In [50]:
import torch
from transformers import AutoTokenizer

def count_matrix_from_alignment(tokenizer, alignment_file):
    alignment_seqs = list(read_multi_fasta(alignment_file).values())
    tokenized_results = tokenizer(alignment_seqs, return_tensors="pt", padding=True, truncation=True)
    alignment_ids = tokenized_results["input_ids"][:,1:-1]
    # count distribution of each column, [seq_len, vocab_size]
    count_matrix = torch.zeros(alignment_ids.size(1), tokenizer.vocab_size)
    for i in range(alignment_ids.size(1)):
        count_matrix[i] = torch.bincount(alignment_ids[:,i], minlength=tokenizer.vocab_size)
    count_matrix = count_matrix / count_matrix.sum(dim=1, keepdim=True)
    return count_matrix


tokenizer = AutoTokenizer.from_pretrained("AI4Protein/ProSST-2048", do_lower_case=False)
alignment_file = "data/proteingym_v1/residue_alignment/A0A1I9GEU1_NEIME_Kennouche_2019.a2m"
logits = count_matrix_from_alignment(tokenizer, alignment_file)
logits

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 7.2033e-04, 2.7895e-01,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 3.6017e-04, 1.8008e-04,
         0.0000e+00],
        ...,
        [9.1410e-01, 0.0000e+00, 1.5487e-02,  ..., 0.0000e+00, 2.6472e-02,
         0.0000e+00],
        [9.2959e-01, 0.0000e+00, 2.6472e-02,  ..., 0.0000e+00, 4.3400e-02,
         0.0000e+00],
        [9.5606e-01, 0.0000e+00, 4.3400e-02,  ..., 0.0000e+00, 3.6017e-04,
         0.0000e+00]])

In [52]:
logits[0], logits[1], sum(logits[0]), sum(logits[1]), logits.size()

(tensor([0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 7.2033e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 7.2033e-04, 2.7895e-01,
         0.0000e+00]),
 tensor([0.0000e+00, 0.0000e+00, 0.0000e+00, 1.1183e-01, 5.2224e-03, 1.2372e-01,
         1.0445e-02, 1.2606e-03, 3.6017e-04, 0.0000e+00, 3.4216e-03, 0.0000e+00,
         2.1610e-03, 6.8431e-03, 1.9809e-03, 1.8008e-04, 3.6017e-04, 5.4025e-04,
         8.2838e-03, 7.1853e-01, 4.8622e-03, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00]),
 tensor(1.),
 tensor(1.),
 torch.Size([161, 25]))

In [34]:
list(read_multi_fasta("data/proteingym_v1/residue_alignment/A0A1I9GEU1_NEIME_Kennouche_2019.a2m").values())

['FTLIELMIVIAIVGILAAVALPAYQDYTARAQVSEAILLAEGQKSAVTEYYLNHGEWPGDNSSAGVATSADIKGKYVQSVTVANGVITAQMASSNVNNEIKSKKLSLWAKRQNGSVKWFCGQPVTrttatatdvaaangktdDKINTKHLPSTCRddssas',
 '----------------------------ARAQVSEAILLAEGQKSAVTEYYLNHGKWPENNDSAGVASASKIIGKYVKSVTVTNGVVTAQMKPSGVNNEIKDKRLSLWAKREDGSVKWFCGQPVKrdnvaaadddvtddknnNGIDTKHLPSTCRdkssa.',
 '----------------------------ARAQVSEAILLAEGQKSAVTEYYLNHGKWPENNGDAGVASASKIIGKYVKQVEVKNGVVTATMNSSNVNKEIQGKRLSLWAKRQDGSVKWFCGQPVK..rnanakddavtadkdKEIETKHLPSTCRdes...',
 'FTLIELMIVIAIVGILAAVALPAYQDYTARAQVSEAILLAEGQKSAVTEYYLNHGIWPENNPA-GVASASDIKGKYVQSVTVANGVVTAQMKSDGVNKEIKNKKLSLWARREAGSVKWFCGQPVTrdnagtdavtadttgkdKEIDTKHLPSTCR......',
 '----------------------------ARAQVSEAILLAEGQKSAVTEYYLNHGEWPENNTSAGVASADKIKGKYVQKVEVAKGVVTAQMASSNVNKEIKDKKLSLWARREDGSVKWFCGQPVTrgagnkaddvtkagndnEKINTKHLPSTCRdkst..',
 '----------------------------ARAQVSEAILLAEGQKSAVTGYYLNHGEWPKDNTSAGVASASNIKGKYVQKVEVNNGVVTATMASSNVNKEIKDKRLSLWAKRQDGSVKWFCGQPVK..rnanakddavtadkdKEIETKHLPSTCRde....',
 '--

In [20]:
logits[-0]

tensor([10.9588, 10.9588, 10.9588, 10.9588, 10.9588, 10.9588, 10.9588, 10.9588,
        10.9588, 10.9588, 10.9588, 10.9588, 10.9588, 10.9588, 10.9588, 10.9588,
        10.9588, 10.9588, 10.9588, 10.9588, 10.9588, 10.9588, 10.9588, 10.9588,
        10.9588])